In [18]:
import pandas as pd 
import numpy as np 
import geopandas as gp 
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew #for some statistics
from math import log
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression as lm
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb


In [8]:
abspath = '/home/semipro321/Documents/Workspace/Geospatial-Analysis-on-House-Price-Prediction/'

In [81]:
df = pd.read_csv(abspath + 'Inputs/mastermodel.csv')
df.info();
df.columns[df.isna().any()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15234 entries, 0 to 15233
Data columns (total 86 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   lat                         15234 non-null  float64
 1   long                        15234 non-null  float64
 2   sqft                        10713 non-null  float64
 3   parking                     15234 non-null  int64  
 4   mean_district_income        15234 non-null  int64  
 5   bedrooms_bg                 15234 non-null  int64  
 6   bedrooms_ag                 15234 non-null  int64  
 7   bathrooms                   15234 non-null  int64  
 8   final_price                 15234 non-null  int64  
 9   type_Att/Row/Twnhouse       15234 non-null  int64  
 10  type_Co-Op Apt              15234 non-null  int64  
 11  type_Co-Ownership Apt       15234 non-null  int64  
 12  type_Comm Element Condo     15234 non-null  int64  
 13  type_Condo Apt              152

Index(['sqft'], dtype='object')

Lets fix sqft

In [10]:
for i,j in enumerate(df.columns):
    print(f'index:{i}, name: {j}')

index:0, name: lat
index:1, name: long
index:2, name: sqft
index:3, name: parking
index:4, name: mean_district_income
index:5, name: bedrooms_bg
index:6, name: bedrooms_ag
index:7, name: bathrooms
index:8, name: final_price
index:9, name: type_Att/Row/Twnhouse
index:10, name: type_Co-Op Apt
index:11, name: type_Co-Ownership Apt
index:12, name: type_Comm Element Condo
index:13, name: type_Condo Apt
index:14, name: type_Condo Townhouse
index:15, name: type_Detached
index:16, name: type_Link
index:17, name: type_Plex
index:18, name: type_Semi-Detached
index:19, name: type_Store W/Apt/Offc
index:20, name: geometry
index:21, name: neighbourhood
index:22, name: dist_nearest_school
index:23, name: dist_nearest_ttc
index:24, name: Assault_2014_500m
index:25, name: Assault_2015_500m
index:26, name: Assault_2016_500m
index:27, name: Assault_2017_500m
index:28, name: Assault_2018_500m
index:29, name: Assault_2019_500m
index:30, name: Auto Theft_2014_500m
index:31, name: Auto Theft_2015_500m
index

In [40]:
df_sqft = df.iloc[:, 0:21].dropna()
df_sqft.drop(columns='geometry', inplace=True)
#filtering detached and semi
df_sqft = df_sqft[(df_sqft.type_Detached == 1) | df_sqft['type_Semi-Detached'] == 1]

In [45]:
X = df_sqft.drop('sqft', axis=1).values
y = df_sqft['sqft'].values

In [42]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression as lm
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb


resultlm = []
for k in [5,10,15,20]:
    crossvalidation = KFold(n_splits=k)
    model = lm().fit(X,y)
    scores = cross_val_score(model, X, y, scoring="neg_root_mean_squared_error", cv=crossvalidation)
    resultlm.append(np.mean(np.abs(scores)))

resultrf = []
for k in [5,10,15,20]:
    crossvalidation = KFold(n_splits=k)
    model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
    scores = cross_val_score(model_xgb, X, y, scoring="neg_root_mean_squared_error", cv=crossvalidation)
    resultrf.append(np.mean(np.abs(scores)))



[18:56:34] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1614844506100/work/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[18:56:36] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1614844506100/work/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[18:56:37] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1614844506100/work/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some par

KeyboardInterrupt: 

In [15]:
resultlm

[0.22762712868027118,
 0.22736096328556415,
 0.22754648228184005,
 0.2275366976582421]

In [16]:
resultrf

[0.2013008137172104,
 0.20158688092604865,
 0.20243829858568907,
 0.20132299701000758]

In [15]:
df2 = df.iloc[:, 0:22].dropna()

In [54]:
df2 = df.iloc[:, 0:22]
df2.drop('geometry', axis=1,inplace=True)
df2 = pd.get_dummies(df2, columns=['neighbourhood'])
X = df2.dropna().drop('sqft', axis=1).values
y = df2.dropna()['sqft'].values
model_xgb = xgb.XGBRegressor()
model_xgb.fit(X, y)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [55]:
X_test = df2[df2.sqft.isna()].drop('sqft', axis=1).values

In [56]:
X_test

array([[ 43.6469817, -79.5303556,   6.       , ...,   0.       ,
          0.       ,   0.       ],
       [ 43.5970962, -79.5096118,   1.       , ...,   0.       ,
          0.       ,   0.       ],
       [ 43.6645415, -79.4614763,   1.       , ...,   0.       ,
          0.       ,   0.       ],
       ...,
       [ 43.7934123, -79.3000096,   4.       , ...,   0.       ,
          0.       ,   0.       ],
       [ 43.7882583, -79.1603421,   1.       , ...,   0.       ,
          0.       ,   0.       ],
       [ 43.6912958, -79.268417 ,   4.       , ...,   0.       ,
          0.       ,   0.       ]])

In [57]:
xgb_pred = model_xgb.predict(X_test)

In [59]:
xgb_pred.shape

(4521,)

In [61]:
df[df.sqft.isna()].shape

(4521, 86)

<ipython-input-63-1df955b316c0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df.sqft.isna()]['sqft'] = pd.Series(xgb_pred)


In [70]:
df_empty = df[df.sqft.isna()]

In [93]:
dfemptyindex=df.loc[df['sqft'].isna(), 'sqft'].index

In [94]:
dfemptyindex

Int64Index([    1,     4,    10,    16,    18,    19,    25,    27,    34,
               42,
            ...
            15209, 15210, 15212, 15213, 15217, 15218, 15223, 15226, 15231,
            15233],
           dtype='int64', length=4521)

In [100]:
predseries = pd.Series(xgb_pred)
predseries.index = dfemptyindex

In [105]:
df.merge(predseries.rename('new'),left_index=True)

MergeError: Can only pass argument "on" OR "left_index" and "right_index", not a combination of both.

In [107]:
df.loc[df['sqft'].isna(), 'sqft'] = predseries

In [111]:
df.loc[df['sqft'].isna(), 'sqft'].shape

(0,)

In [113]:
df.sqft.isna().any()

False

In [114]:
df.to_csv('mastermodel_filled.csv', index=False)